## Prepare the workspace

In [1]:
# Check torch version and CUDA status if GPU is enabled.
import torch
print(torch.__version__)
print(torch.cuda.is_available()) # Should return True when GPU is enabled. 

2.4.1
False


# Developing an AI application

Going forward, AI algorithms will be incorporated into more and more everyday applications. For example, you might want to include an image classifier in a smart phone app. To do this, you'd use a deep learning model trained on hundreds of thousands of images as part of the overall application architecture. A large part of software development in the future will be using these types of models as common parts of applications. 

In this project, you'll train an image classifier to recognize different species of flowers. You can imagine using something like this in a phone app that tells you the name of the flower your camera is looking at. In practice you'd train this classifier, then export it for use in your application. We'll be using [this dataset](http://www.robots.ox.ac.uk/~vgg/data/flowers/102/index.html) of 102 flower categories, you can see a few examples below. 

<img src='assets/Flowers.png' width=500px>

The project is broken down into multiple steps:

* Load and preprocess the image dataset
* Train the image classifier on your dataset
* Use the trained classifier to predict image content

We'll lead you through each part which you'll implement in Python.

When you've completed this project, you'll have an application that can be trained on any set of labeled images. Here your network will be learning about flowers and end up as a command line application. But, what you do with your new skills depends on your imagination and effort in building a dataset. For example, imagine an app where you take a picture of a car, it tells you what the make and model is, then looks up information about it. Go build your own dataset and make something new.

First up is importing the packages you'll need. It's good practice to keep all the imports at the beginning of your code. As you work through this notebook and find you need to import a package, make sure to add the import up here.

In [2]:
# Imports here
import torch
import numpy as np
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import seaborn as sns
import os

from torchvision import datasets, transforms, models
from collections import OrderedDict
from PIL import Image
from tqdm import tqdm


# Check if MPS (Metal Performance Shaders) is available for MacBook Pro M1
if torch.backends.mps.is_available():
    device = torch.device('mps')
elif torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

print(f"Device: {device}")

# Verify that the necessary packages are installed
try:
    import torch
    import numpy as np
    import torch.nn as nn
    import torch.optim as optim
    from torchvision import datasets, transforms, models
    from collections import OrderedDict
    from PIL import Image
    import matplotlib.pyplot as plt
    import seaborn as sns
except ImportError as e:
    print(f"Error importing packages: {e}")
    raise

# Example usage of the device
# This is just a placeholder to show how you might use the device in practice
# Replace this with your actual model and training code
x = torch.rand(5, 3).to(device)
print(f"Tensor on {device}: {x}")

Device: mps
Tensor on mps: tensor([[0.9384, 0.4814, 0.7049],
        [0.0900, 0.0579, 0.3974],
        [0.4490, 0.5453, 0.7101],
        [0.8957, 0.0380, 0.6916],
        [0.0622, 0.4289, 0.9008]], device='mps:0')


## Load the data

Here you'll use `torchvision` to load the data ([documentation](http://pytorch.org/docs/0.3.0/torchvision/index.html)). The data should be included alongside this notebook, otherwise you can [download it here](https://s3.amazonaws.com/content.udacity-data.com/nd089/flower_data.tar.gz). 

If you do not find the `flowers/` dataset in the current directory, **/workspace/home/aipnd-project/**, you can download it using the following commands. 

**Method 1**
```bash
!wget 'https://s3.amazonaws.com/content.udacity-data.com/nd089/flower_data.tar.gz'
!unlink flowers
!mkdir flowers && tar -xzf flower_data.tar.gz -C flowers
```


**Method 2**
```bash
!cp -r /data/ .
```

### If your device is Mac OS M1, you can use the following command to install wget to download the dataset

In [3]:
# If you are using a MacBook Pro M1, you may need to install wget
!brew install wget

==> Downloading https://formulae.brew.sh/api/formula.jws.json
######################################################################### 100.0%
==> Downloading https://formulae.brew.sh/api/cask.jws.json
######################################################################### 100.0%
To reinstall 1.24.5, run:
  brew reinstall wget


### Check if folder and file exists or not to avoid redownloading the dataset

In [4]:
# Check if file not exists, download and unzip the file
if not os.path.exists("flower_data.tar.gz"):
    # Download the file
    !wget 'https://s3.amazonaws.com/content.udacity-data.com/nd089/flower_data.tar.gz'
    # Unzip the file
    !unzip flower_data.tar.gz -d flowers
else:
    # Check if flower directory not exists
    if not os.path.exists("flowers"):
        # Unzip the file
        !mkdir flowers && tar -xvf flower_data.tar.gz -C flowers
        !ls flowers

## Data Description
The dataset is split into three parts, training, validation, and testing. For the training, you'll want to apply transformations such as random scaling, cropping, and flipping. This will help the network generalize leading to better performance. You'll also need to make sure the input data is resized to 224x224 pixels as required by the pre-trained networks.

The validation and testing sets are used to measure the model's performance on data it hasn't seen yet. For this you don't want any scaling or rotation transformations, but you'll need to resize then crop the images to the appropriate size.

The pre-trained networks you'll use were trained on the ImageNet dataset where each color channel was normalized separately. For all three sets you'll need to normalize the means and standard deviations of the images to what the network expects. For the means, it's `[0.485, 0.456, 0.406]` and for the standard deviations `[0.229, 0.224, 0.225]`, calculated from the ImageNet images.  These values will shift each color channel to be centered at 0 and range from -1 to 1.
 

In [5]:
data_dir = 'flowers'
train_dir = data_dir + '/train'
valid_dir = data_dir + '/valid'
test_dir = data_dir + '/test'

In [6]:
# TODO: Define your transforms for the training, validation, and testing sets
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomRotation(30),
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'valid': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

# TODO: Load the datasets with ImageFolder
image_datasets = {
    'train': datasets.ImageFolder(train_dir, transform=data_transforms['train']),
    'valid': datasets.ImageFolder(valid_dir, transform=data_transforms['valid']),
    'test': datasets.ImageFolder(test_dir, transform=data_transforms['test'])
}

# TODO: Using the image datasets and the trainforms, define the dataloaders
dataloaders = {
    'train': torch.utils.data.DataLoader(image_datasets['train'], batch_size=64, shuffle=True),
    'valid': torch.utils.data.DataLoader(image_datasets['valid'], batch_size=64),
    'test': torch.utils.data.DataLoader(image_datasets['test'], batch_size=64)
}

### Label mapping

You'll also need to load in a mapping from category label to category name. You can find this in the file `cat_to_name.json`. It's a JSON object which you can read in with the [`json` module](https://docs.python.org/2/library/json.html). This will give you a dictionary mapping the integer encoded categories to the actual names of the flowers.

In [7]:
import json

with open('cat_to_name.json', 'r') as f:
    cat_to_name = json.load(f)

# Building and training the classifier

Now that the data is ready, it's time to build and train the classifier. As usual, you should use one of the pretrained models from `torchvision.models` to get the image features. Build and train a new feed-forward classifier using those features.

We're going to leave this part up to you. Refer to [the rubric](https://review.udacity.com/#!/rubrics/1663/view) for guidance on successfully completing this section. Things you'll need to do:

* Load a [pre-trained network](http://pytorch.org/docs/master/torchvision/models.html) (If you need a starting point, the VGG networks work great and are straightforward to use)
* Define a new, untrained feed-forward network as a classifier, using ReLU activations and dropout
* Train the classifier layers using backpropagation using the pre-trained network to get the features
* Track the loss and accuracy on the validation set to determine the best hyperparameters

We've left a cell open for you below, but use as many as you need. Our advice is to break the problem up into smaller parts you can run separately. Check that each part is doing what you expect, then move on to the next. You'll likely find that as you work through each part, you'll need to go back and modify your previous code. This is totally normal!

When training make sure you're updating only the weights of the feed-forward network. You should be able to get the validation accuracy above 70% if you build everything right. Make sure to try different hyperparameters (learning rate, units in the classifier, epochs, etc) to find the best model. Save those hyperparameters to use as default values in the next part of the project.

One last important tip if you're using the workspace to run your code: To avoid having your workspace disconnect during the long-running tasks in this notebook, please read in the earlier page in this lesson called Intro to
GPU Workspaces about Keeping Your Session Active. 

## Note for Workspace users: 
If your network is over 1 GB when saved as a checkpoint, there might be issues with saving backups in your workspace. Typically this happens with wide dense layers after the convolutional layers. If your saved checkpoint is larger than 1 GB (you can open a terminal and check with `ls -lh`), you should reduce the size of your hidden layers and train again.

In [8]:
# TODO: Build and train your network
def model_setup(structure = 'vgg16', dropout = 0.2, hidden_layer = 4096, lr = 0.001):
    """
    Setup the model, optimizer and criterion.

    Args:
    device (torch.device): The device to use.
    structure (str): The model architecture to use. Default is 'vgg16'.
    hidden_layer (int): The number of hidden layers. Default is 4096.
    lr (float): The learning rate for the optimizer. Default is 0.001.
    dropout (float): The dropout rate. Default is 0.2.

    Returns:
    model (torch.nn.Module): The model to use.
    optimizer (torch.optim): The optimizer to use.
    criterion (torch.nn): The criterion to use.
    """
    structures = {"vgg16": 25088, "alexnet": 9216, "resnet18": 512, "densenet121": 1024}

    if structure == "vgg16":
        model = models.vgg16(weights=models.VGG16_Weights.IMAGENET1K_V1)
    elif structure == "alexnet":
        model = models.alexnet(weights=models.AlexNet_Weights.IMAGENET1K_V1)
    elif structure == "resnet18":
        model = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)
    elif structure == "densenet121":
        model = models.densenet121(weights=models.DenseNet121_Weights.IMAGENET1K_V1)
    else:
        raise ValueError(
            f"The {structure} is not a valid model. Please input either 'vgg16', 'alexnet', 'resnet18', or 'densenet121'!"
        )

    print(f"Using {structure} model with pre-trained weights.")

    for param in model.parameters():
        param.requires_grad = False

    if structure in ["vgg16", "alexnet"]:
        classifier = nn.Sequential(
            OrderedDict(
                [
                    ("dropout1", nn.Dropout(dropout)),
                    ("fc1", nn.Linear(structures[structure], hidden_layer)),
                    ("relu1", nn.ReLU()),
                    ("fc2", nn.Linear(hidden_layer, 1024)),
                    ("relu2", nn.ReLU()),
                    ("fc3", nn.Linear(1024, 102)),
                    ("output", nn.LogSoftmax(dim=1)),
                ]
            )
        )
        model.classifier = classifier
    elif structure == "resnet18":
        model.fc = nn.Sequential(
            OrderedDict(
                [
                    ("dropout1", nn.Dropout(dropout)),
                    ("fc1", nn.Linear(structures[structure], hidden_layer)),
                    ("relu1", nn.ReLU()),
                    ("fc2", nn.Linear(hidden_layer, 102)),
                    ("output", nn.LogSoftmax(dim=1)),
                ]
            )
        )
    elif structure == "densenet121":
        model.classifier = nn.Sequential(
            OrderedDict(
                [
                    ("dropout1", nn.Dropout(dropout)),
                    ("fc1", nn.Linear(structures[structure], hidden_layer)),
                    ("relu1", nn.ReLU()),
                    ("fc2", nn.Linear(hidden_layer, 102)),
                    ("output", nn.LogSoftmax(dim=1)),
                ]
            )
        )

    criterion = nn.NLLLoss()
    optimizer = optim.Adam(
        (
            model.classifier.parameters()
            if structure in ["vgg16", "alexnet", "densenet121"]
            else model.fc.parameters()
        ),
        lr,
    )

    model.to(device)

    print("Model setup complete.")

    return model, optimizer, criterion

In [9]:
model, optimizer, criterion = model_setup()
print(model)

Using vgg16 model with pre-trained weights.
Model setup complete.
VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)


In [10]:
# Validate the model with the validation dataset to check the accuracy and loss
def validate(device, model, data_loader, criterion):
    """
    Validate the model with the validation dataset to check the accuracy and loss.

    Args:
    device (torch.device): The device to use.
    model (torch.nn.Module): The model to use.
    data_loader (torch.utils.data.DataLoader): The data loader to use.
    criterion (torch.nn): The criterion to use.

    Returns:
    avg_val_loss (float): The average validation loss.
    avg_val_accuracy (float): The average validation accuracy.
    """
    val_loss = 0
    val_accuracy = 0
    total = 0

    with torch.no_grad():
        model.eval()
        for images, labels in tqdm(data_loader, desc="Validating"):
            images, labels = images.to(device), labels.to(device)

            outputs = model(images)

            loss = criterion(outputs, labels)
            val_loss += loss.item()

            ps = torch.exp(outputs)
            _, indices = ps.topk(1, dim=1)
            equality = indices == labels.view(*indices.shape)
            val_accuracy += torch.sum(equality.type(torch.FloatTensor)).item()
            total += labels.size(0)

    avg_val_loss = val_loss / len(data_loader)
    avg_val_accuracy = val_accuracy / total

    print(
        f"Validation Loss: {avg_val_loss:.4f}, Validation Accuracy: {avg_val_accuracy:.4f}"
    )

    return avg_val_loss, avg_val_accuracy

In [11]:
# Train the model with the training dataset
def train_model(
    device,
    dataloaders,
    image_datasets,
    model,
    optimizer,
    criterion,
    epochs=5,
    print_every=50,
):
    """
    Train the model with the training dataset.

    Args:
    device (torch.device): The device to use.
    dataloaders (dict): The data loaders to use.
    image_datasets (dict): The image datasets to use.
    model (torch.nn.Module): The model to use.
    optimizer (torch.optim): The optimizer to use.
    criterion (torch.nn): The criterion to use.
    epochs (int): The number of epochs to train the model. Default is 5.
    print_every (int): The number of steps to print the training loss and validation loss. Default is 50.
    """
    print("Training process is now starting...")
    print(f"Device: {device}")

    if image_datasets is None:
        print("Image datasets is None. Please initialize the image datasets.")
        return

    # Check if dataloaders and image_datasets are not None
    if dataloaders is None:
        print("Dataloaders is None. Please initialize the data loaders.")
        return

    # Check if 'train' and 'valid' keys exist in dataloaders
    if "train" not in dataloaders or "valid" not in dataloaders:
        print("'train' or 'valid' key is missing in dataloaders.")
        return

    model.train()
    steps = 0
    running_loss = 0
    val_len = len(dataloaders["valid"])

    for epoch in range(epochs):
        for images, labels in tqdm(
            dataloaders["train"], desc="Epoch: {}/{}".format(epoch + 1, epochs)
        ):
            steps += 1

            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()

            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

            if steps % print_every == 0:
                val_loss, val_accuracy = validate(
                    device, model, dataloaders["valid"], criterion
                )

                # Logging above print statement
                print(
                    f"Epoch: {epoch + 1}/{epochs}.."
                    f"Training loss: {running_loss/print_every:.3f}.."
                    f"Validation loss: {val_loss/val_len:.3f}.."
                    f"Validation accuracy: {val_accuracy/val_len:.3f}"
                )
                running_loss = 0
                model.train()
    model.class_to_idx = image_datasets["train"].class_to_idx
    print("Training process is now complete!")


In [12]:
# Run the training process
train_model(device, dataloaders, image_datasets, model, optimizer, criterion, epochs=5, print_every=50)


Training process is now starting...
Device: mps


Epoch: 1/5:  11%|█         | 11/103 [02:13<18:22, 11.98s/it]

## Testing your network

It's good practice to test your trained network on test data, images the network has never seen either in training or validation. This will give you a good estimate for the model's performance on completely new images. Run the test images through the network and measure the accuracy, the same way you did validation. You should be able to reach around 70% accuracy on the test set if the model has been trained well.

In [51]:
# TODO: Do validation on the test set
def test_model(device, model, criterion, dataloader):
    """
    Test the model with the test dataset.

    Args:
    device (torch.device): The device to use.
    model (torch.nn.Module): The model to use.
    criterion (torch.nn): The criterion to use.
    dataloader (torch.utils.data.DataLoader): The data loader to use.

    Returns:
    test_loss (float): The test loss.
    test_accuracy (float): The test accuracy.
    """
    test_loss = 0
    test_accuracy = 0

    with torch.no_grad():
        model.eval()
        for images, labels in dataloader:
            images, labels = images.to(device), labels.to(device)
            
            outputs = model(images)
            
            test_loss += criterion(outputs, labels)
            
            ps = torch.exp(outputs)
            probs, indices = ps.topk(1, dim = 1)
            equalilty = indices == labels.view(*indices.shape)
            test_accuracy += torch.mean(equalilty.type(torch.FloatTensor)).item()

    return test_loss, test_accuracy

In [ ]:
# Run the test process
test_loss, test_accuracy = test_model(device, model, criterion, dataloaders['test'])
print(f"Test loss: {test_loss/len(dataloaders['test']):.3f}.."
     f"Test accuracy: {test_accuracy/len(dataloaders['test']):.3f}")

## Save the checkpoint

Now that your network is trained, save the model so you can load it later for making predictions. You probably want to save other things such as the mapping of classes to indices which you get from one of the image datasets: `image_datasets['train'].class_to_idx`. You can attach this to the model as an attribute which makes inference easier later on.

```model.class_to_idx = image_datasets['train'].class_to_idx```

Remember that you'll want to completely rebuild the model later so you can use it for inference. Make sure to include any information you need in the checkpoint. If you want to load the model and keep training, you'll want to save the number of epochs as well as the optimizer state, `optimizer.state_dict`. You'll likely want to use this trained model in the next part of the project, so best to save it now.

In [53]:
# TODO: Save the checkpoint 
def save_checkpoint(
    model,
    optimizer,
    structure,
    image_datasets,
    save_dir="checkpoint.pth",
    hidden_layer=4096,
):
    """
    Save the checkpoint of the model.

    Args:
    model (torch.nn.Module): The model to save.
    optimizer (torch.optim): The optimizer to save.
    structure (str): The model architecture to save.
    image_datasets (dict): The image datasets to save.
    save_dir (str): The directory to save the checkpoint. Default is 'checkpoint.pth'.
    hidden_layer (int): The number of hidden layers. Default is 4096.
    """
    model.class_to_idx = image_datasets["train"].class_to_idx
    checkpoint = {
        "structure": structure,
        "hidden_layer": hidden_layer,
        "state_dict": model.state_dict(),
        "class_to_idx": model.class_to_idx,
        "optimizer": optimizer.state_dict(),
    }

    torch.save(checkpoint, save_dir)
    print(f"Checkpoint saved to {save_dir}")

In [ ]:
# Save the checkpoint
save_checkpoint(model, optimizer, 'vgg16', image_datasets, 'checkpoint.pth')

## Loading the checkpoint

At this point it's good to write a function that can load a checkpoint and rebuild the model. That way you can come back to this project and keep working on it without having to retrain the network.

In [55]:
# TODO: Write a function that loads a checkpoint and rebuilds the model
def load_checkpoint(filepath, device):
    """
    Load the checkpoint and rebuild the model.

    Args:
    filepath (str): The path to the checkpoint.
    device (torch.device): The device to use.

    Returns:
    model (torch.nn.Module): The model to use.
    optimizer (torch.optim): The optimizer to use.
    criterion (torch.nn): The criterion to use.
    """
    checkpoint = torch.load(filepath, weights_only=True)
    
    structure = checkpoint['structure']
    model, optimizer, criterion = model_setup()
    
    model.class_to_idx = checkpoint['class_to_idx']
    model.load_state_dict(checkpoint['state_dict'])
    
    return model, optimizer, criterion

In [ ]:
# Load the checkpoint
model, optimizer, criterion = load_checkpoint('checkpoint.pth', device)
print(model)

# Inference for classification

Now you'll write a function to use a trained network for inference. That is, you'll pass an image into the network and predict the class of the flower in the image. Write a function called `predict` that takes an image and a model, then returns the top $K$ most likely classes along with the probabilities. It should look like 

```python
probs, classes = predict(image_path, model)
print(probs)
print(classes)
> [ 0.01558163  0.01541934  0.01452626  0.01443549  0.01407339]
> ['70', '3', '45', '62', '55']
```

First you'll need to handle processing the input image such that it can be used in your network. 

## Image Preprocessing

You'll want to use `PIL` to load the image ([documentation](https://pillow.readthedocs.io/en/latest/reference/Image.html)). It's best to write a function that preprocesses the image so it can be used as input for the model. This function should process the images in the same manner used for training. 

First, resize the images where the shortest side is 256 pixels, keeping the aspect ratio. This can be done with the [`thumbnail`](http://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.thumbnail) or [`resize`](http://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.thumbnail) methods. Then you'll need to crop out the center 224x224 portion of the image.

Color channels of images are typically encoded as integers 0-255, but the model expected floats 0-1. You'll need to convert the values. It's easiest with a Numpy array, which you can get from a PIL image like so `np_image = np.array(pil_image)`.

As before, the network expects the images to be normalized in a specific way. For the means, it's `[0.485, 0.456, 0.406]` and for the standard deviations `[0.229, 0.224, 0.225]`. You'll want to subtract the means from each color channel, then divide by the standard deviation. 

And finally, PyTorch expects the color channel to be the first dimension but it's the third dimension in the PIL image and Numpy array. You can reorder dimensions using [`ndarray.transpose`](https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.ndarray.transpose.html). The color channel needs to be first and retain the order of the other two dimensions.

In [57]:
def process_image(image_path):
    """
    Process an image for use in a PyTorch model.

    Args:
    image_path (str): The path to the image.

    Returns:
    img (torch.Tensor): The processed image as a PyTorch tensor.
    """
    
    # TODO: Process a PIL image for use in a PyTorch model
    img = Image.open(image_path)
    img_transforms = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

    img = img_transforms(img)

    return img

To check your work, the function below converts a PyTorch tensor and displays it in the notebook. If your `process_image` function works, running the output through this function should return the original image (except for the cropped out portions).

In [58]:
def imshow(image, ax=None, title=None):
    """
    Imshow for Tensor.

    Args:
    image (torch.Tensor): The image to display.
    ax (matplotlib.axes.Axes): The axes to display the image.
    title (str): The title of the image.

    Returns:
    ax (matplotlib.axes.Axes): The axes to display the image.
    """
    if ax is None:
        fig, ax = plt.subplots()

    # PyTorch tensors assume the color channel is the first dimension
    image = image.numpy().transpose((1, 2, 0))

    # Undo preprocessing
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    image = std * image + mean

    # Image needs to be clipped between 0 and 1 or it looks like noise when displayed
    image = np.clip(image, 0, 1)

    # Display the image
    ax.imshow(image)

    return ax

In [59]:
# Test the process_image function
img = process_image('flowers/test/1/image_06743.jpg')

## Class Prediction

Once you can get images in the correct format, it's time to write a function for making predictions with your model. A common practice is to predict the top 5 or so (usually called top-$K$) most probable classes. You'll want to calculate the class probabilities then find the $K$ largest values.

To get the top $K$ largest values in a tensor use [`x.topk(k)`](http://pytorch.org/docs/master/torch.html#torch.topk). This method returns both the highest `k` probabilities and the indices of those probabilities corresponding to the classes. You need to convert from these indices to the actual class labels using `class_to_idx` which hopefully you added to the model or from an `ImageFolder` you used to load the data ([see here](#Save-the-checkpoint)). Make sure to invert the dictionary so you get a mapping from index to class as well.

Again, this method should take a path to an image and a model checkpoint, then return the probabilities and classes.

```python
probs, classes = predict(image_path, model)
print(probs)
print(classes)
> [ 0.01558163  0.01541934  0.01452626  0.01443549  0.01407339]
> ['70', '3', '45', '62', '55']
```

In [60]:
def predict(image_path, model, device, cat_to_name, topk=5):
    """
    Predict the class of an image.

    Args:
    image_path (str): The path to the image.
    model (torch.nn.Module): The model to use.
    device (torch.device): The device to use.
    cat_to_name (dict): The category to name mapping.
    topk (int): The number of top classes to return. Default is 5.

    Returns:
    probs (list): The probabilities of the top classes.
    classes (list): The classes of the top classes.
    flowers (list): The names of the top classes.
    """
    model.eval()
    img = process_image(image_path)

    # Convert 2D image to 1D vector
    with torch.no_grad():
        # Convert 2D image to 1D vector
        img = img.unsqueeze(0)
        img = img.to(device)

        output = model(img)
        ps = torch.exp(output)
        probs, indices = ps.topk(topk)

        # Convert indices to classes
        probs = probs.cpu().numpy().tolist()[0]
        indices = indices.cpu().numpy().tolist()[0]

        # Invert the class_to_idx dictionary
        idx_to_class = {val: key for key, val in model.class_to_idx.items()}
        classes = [idx_to_class[idx] for idx in indices]
        flowers = [cat_to_name[cls] for cls in classes]

    return probs, classes, flowers

In [ ]:
# Test the predict function
probs, classes, flowers = predict('flowers/test/1/image_06752.jpg', model, device, cat_to_name, topk=5)

print(probs)
print(classes)
print(flowers)

## Sanity Checking

Now that you can use a trained model for predictions, check to make sure it makes sense. Even if the testing accuracy is high, it's always good to check that there aren't obvious bugs. Use `matplotlib` to plot the probabilities for the top 5 classes as a bar graph, along with the input image. It should look like this:

<img src='assets/inference_example.png' width=300px>

You can convert from the class integer encoding to actual flower names with the `cat_to_name.json` file (should have been loaded earlier in the notebook). To show a PyTorch tensor as an image, use the `imshow` function defined above.

In [62]:
# TODO: Display an image along with the top 5 classes
def display_image(image_path, model, device, cat_to_name):
    """
    Display the image with the prediction.

    Args:
    image_path (str): The path to the image.
    model (torch.nn.Module): The model to use.
    device (torch.device): The device to use.
    cat_to_name (dict): The category to name mapping.
    """
        
    # Set up the plot
    plt.figure(figsize = (6,10))
    ax = plt.subplot(2,1,1)
    
    # Set up title
    flower_num = image_path.split('/')[2]
    title = cat_to_name[flower_num]
    
    # Plot flower
    img = process_image(image_path)
    imshow(img, ax, title = title);
    
    # Make prediction
    probs, classes, flowers = predict(image_path, model, device, cat_to_name)
    
    # Plot bar chart
    plt.subplot(2,1,2)
    sns.barplot(x=probs, y=flowers, color=sns.color_palette()[0]);
    plt.show()

In [ ]:
# Display the image
display_image('flowers/test/1/image_06752.jpg', model, device, cat_to_name)

## Reminder for Workspace users
If your network becomes very large when saved as a checkpoint, there might be issues with saving backups in your workspace. You should reduce the size of your hidden layers and train again. 
    
We strongly encourage you to delete these large interim files and directories before navigating to another page or closing the browser tab.

In [64]:
# TODO remove .pth files or move it to a temporary `~/opt` directory in this Workspace
!rm checkpoint.pth